In [1]:
%matplotlib inline
import seaborn as sbn
sbn.set_style('white')
import numpy as np
from skimage.io import imsave
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.manifold import TSNE

from keras.models import Model
from keras.layers import InputLayer
from keras import backend as K

from machinedesign.autoencoder.interface import load
from machinedesign.viz import grid_of_images_default
from sklearn.ensemble import RandomForestClassifier

from PIL import Image

import math

from collections import defaultdict
from scipy.spatial.distance import cdist
from lapjv import lapjv

def grid_embedding(h):
    assert int(np.sqrt(h.shape[0])) ** 2 == h.shape[0], 'Nb of examples must be a square number'
    size = np.sqrt(h.shape[0])
    grid = np.dstack(np.meshgrid(np.linspace(0, 1, size), np.linspace(0, 1, size))).reshape(-1, 2)
    cost_matrix = cdist(grid, h, "sqeuclidean").astype('float32')
    cost_matrix = cost_matrix * (100000 / cost_matrix.max())
    _, rows, cols = lapjv(cost_matrix)
    return rows

def convert_to_RGBA(img, alpha=None):
    """Converts input PIL Image or nparray to RGBA. Returns ndarray of shape
    (H, W, 4), where the last channel dimension is `alpha`, the transparency"""
    try:
        if img.mode != "RGBA":
            img = np.array(img.convert("RGBA"))
    except AttributeError:
        img = Image.fromarray(img)
        if img.mode != "RGBA":
            img = np.array(img.convert("RGBA"))
    # reconvert back to npy. we converted ^ to make sure any array is converted
    # to a standard color mode, RGBA, so we can manpipualate easily
    if alpha is not None:
        if alpha >= 0 and alpha <= 255:
            img[:, :, 3] = int(alpha)
        else:
            raise RuntimeError("alpha must be between [0, 255]")

    return np.array(img)


def preproc_img(img, flip_ud=True, alpha=None):
    """ Preprocess a image arr (numpy or PIL.Image) to be absorbed bokeh's
    plotting."""
    # convert to RGBA if it's not, depending on whether ndarr/PIL.Image
    img = convert_to_RGBA(img, alpha=alpha)

    # traditional image conventions refer to top left corner as 0,0.
    # however, bokeh has bottom left (0,0).
    if flip_ud:
        img = np.flipud(img)

    # convert img to be 2d arr (M x N) with dtype = uint32 -- for bokeh
    bimg = np.squeeze(img.astype(np.uint8).view(np.uint32))
    return bimg
def plot_dataset(code_2d, categories):
    colors = [
        'r',
        'b',
        'g',
        'crimson',
        'gold',
        'yellow',
        'maroon',
        'm',
        'c',
        'orange'
    ]
    for cat in range(0, 10):
        g = (categories == cat)
        plt.scatter(
            code_2d[g, 0],
            code_2d[g, 1],
            marker='+',
            c=colors[cat],
            s=40,
            alpha=0.7,
            label="digit {}".format(cat)
        )


def plot_generated(code_2d, categories):
    g = (categories < 0)
    plt.scatter(
        code_2d[g, 0],
        code_2d[g, 1],
        marker='+',
        c='gray',
        s=30
    )
    
def index_of(f, it):
    for i, x in enumerate(it):
        if f(x):
            return i

Using Theano backend.


In [2]:
dataset = '../data/celeba.npz'
data = np.load(dataset)
X = data['X']
#y = data['y']
X = X.astype(np.float32)
X /= 255.

In [3]:
folder = 'celeba_vertebrate_deep'

In [4]:
model = load('results/{}'.format(folder))

In [21]:
# *_vertebrate_* or *_vertebrate_deep_*
inp_layers = [ 
    model.get_layer('winner_take_all_spatial_1'),
    model.get_layer('winner_take_all_spatial_2'),
    model.get_layer('winner_take_all_spatial_3'),
]
inp = [l.output for l in inp_layers]
out = [
    model.get_layer('activation_4').output,
    model.get_layer('activation_6').output,
    model.get_layer('activation_9').output,
    model.get_layer('activation_10').output
]
dec = K.function(inp, out)
enc = K.function([model.layers[0].input], inp)

In [23]:
# *_vertebrate
H = [np.zeros((layer.input_shape[1],) + layer.input_shape[1:]) for layer in inp_layers]
ind = np.arange(inp_layers[0].input_shape[1])
for h in H:
    print(h.shape)
H[0][ind, ind, 2, 2] = 1.0
outs = dec(H)
im = grid_of_images_default(outs[3], normalize=True)
imsave('results/{}/features.png'.format(folder), im)

(256, 256, 16, 16)
(256, 256, 8, 8)
(256, 256, 4, 4)


/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [24]:
h = enc([X[0:64]])
outs = dec(h)
outs = [o[0:1] for o in outs]
o = np.concatenate(outs, axis=0)
print(o.shape)
im = grid_of_images_default(o, normalize=True)
imsave('results/{}/features.png'.format(folder), im)

(4, 3, 32, 32)


/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [ ]:
def featurize(h):
    if len(h.shape) == 4:
        y, x = np.indices((layer_h, layer_w))
        y = y[np.newaxis, np.newaxis, :, :]
        x = x[np.newaxis, np.newaxis, :, :]
        hx = (h > 0) * x 
        hy = (h > 0) * y
        hx = hx.max(axis=(2, 3))
        hy = hy.max(axis=(2, 3))
        hx = hx.astype(np.float32)
        hy = hy.astype(np.float32)
        hy /= layer_h
        hx /= layer_w
        h = np.concatenate((hx, hy), axis=1)
    else:
        h = h.reshape((h.shape[0], -1))
    return h

In [ ]:
gen = np.load('results/{}/gen/generated.npz'.format(folder))
nb_gen = 2 * (2 * 40 * 40)
#nb_gen = 2 * (2 * 15 * 15)
Xg = gen['generated'][0:nb_gen]
yg = np.array([-1] * len(Xg))
Xt = X[0:len(Xg)]
yt = y[0:len(yg)]
Xf = np.concatenate((Xg, Xt), axis=0)
yf = np.concatenate((yg, yt), axis=0)
h = enc([Xf])
print(Xf.shape, h.shape)
h = featurize(h)
h2d = TSNE().fit_transform(h)

In [ ]:
fig = plt.figure(figsize=(15, 15))
#plt.scatter(h2d[yf>=0, 0], h2d[yf>=0, 1], color='blue', marker='+')
#plt.scatter(h2d[yf<0, 0], h2d[yf<0, 1], color='red', marker='+')
plot_dataset(h2d, yf)
plot_generated(h2d, yf)
plt.legend()
plt.show()

In [ ]:
from bokeh.plotting import figure, output_file, show, save
output_file('results/{}/img_embedding.html'.format(folder))

x_range = (h2d[:, 1].min(), h2d[:, 1].max())
y_range = (h2d[:, 0].min(), h2d[:, 1].max())
p = figure(plot_width=1200, plot_height=800, x_range=x_range, y_range=y_range)

for coord, img in zip(h2d, Xf):
    img = img[0]
    img = img * 255.
    img = img.astype(np.float32)
    img = Image.fromarray(img)
    img = preproc_img(img)
    y = coord[0]
    x = coord[1]
    p.image_rgba(image=[img], 
                 x=[y],
                 y=[x], 
                 dw=[0.1], dh=[0.1])

save(p)

In [ ]:
nb = 15*15*2
xx = np.concatenate((Xf[0:nb], Xf[nb_gen:nb_gen + nb]), axis=0)
hh = np.concatenate((h2d[0:nb], h2d[nb_gen:nb_gen + nb]), axis=0)
xx[0:nb] = 1.0 - xx[0:nb]
rows = grid_embedding(hh)
im = grid_of_images_default(xx[rows])
imsave('results/{}/grid.png'.format(folder), im)

In [ ]:
plt.hist(h[yf<0, 2], color='blue', histtype='step')
plt.hist(h[yf>=0, 2], color='red', histtype='step')
plt.show()

In [ ]:
# Computes normalized mutual information between all possible pairs of features
ind = np.arange(h.shape[1]//2, h.shape[1])
np.random.shuffle(ind)
h_ = h[:, ind]
m = np.zeros((h_.shape[1], h_.shape[1]))
for i in range(h_.shape[1]):
    for j in range(i, h_.shape[1]):
        ij_counts = defaultdict(int)
        icounts = defaultdict(int)
        jcounts = defaultdict(int)
        for k in range(0, 1000):
            ij_counts[(h_[k, i], h_[k, j])] += 1
            icounts[h_[k, i]] += 1
            jcounts[h_[k, j]] += 1
        ij_total = sum(ij_counts.values())
        i_total = sum(icounts.values())
        j_total = sum(jcounts.values())
        m[i, j] = 0.
        for (vi, vj), nb in ij_counts.items():
            pij = nb / ij_total
            pi = icounts[vi] / i_total
            pj = jcounts[vj] / j_total
            m[i, j] += pij * math.log(pij / (pi * pj))
        hi = sum((nb/i_total) * math.log(nb/i_total) for nb in icounts.values())
        hj = sum((nb/j_total) * math.log(nb/j_total) for nb in jcounts.values())
        m[i, j] /= math.sqrt(hi * hj)

In [ ]:
fig = plt.figure(figsize=(20, 20))
plt.imshow(m, cmap='plasma')

In [ ]:
x = K.placeholder((None, 1, 28, 28))
layers = model.layers[2].layers
#layers = model.layers
y = x
for t in range(20):
    for l in layers:
        y = l(y)
loss = y[:, :, 13, 13].mean()
g = K.gradients(loss, x)
get_g = K.function([x], g)
pred = K.function([x], y)

In [ ]:
v = np.random.uniform(size=(1, 1, 28, 28)).astype(np.float32)
#v = np.ones((1, 1, 28, 28)).astype(np.float32)
#v = X[0:1]
gval = (get_g([v])).astype(np.float32)
plt.imshow(gval[0, 0], cmap='gray')
plt.show()

In [ ]:
v = np.random.uniform(size=(1, 1, 28, 28)).astype(np.float32)
y=pred([v])
plt.imshow(y[0,0], cmap='gray')
plt.show()
plt.imshow(X[0,0], cmap='gray')
plt.show()

In [ ]:
h = enc([X[0:1]])
(h == 0).sum() #/ h.shape[1]

In [ ]:
(h[0]>0).sum()